In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Graph execution error
#!pip install tensorflow==2.7

In [ ]:
#!pip install tensorflow

In [2]:
#. モジュールの読み込み
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#. データの読み込み
df = pd.read_csv("/content/drive/MyDrive/datasienceブログ/221106_lyrics_data.csv", encoding="utf-8")

In [4]:
#. データの確認
df.head(3)

,artist,song,lyrics,flg
0,I Don't Like Monday,ダイナマイト,ループしてる同じ 毎日を繰り返し まるで機械みたい 息を止め動いてる 車の中レディオ 音楽が...,0
1,I Don't Like Monday,愛言葉,出逢った頃とおんなじ様な 馬鹿みたく晴れた空だ 並んで歩いた緑道は 微かに夏の匂いがした ...,1
2,I Don't Like Monday,AITAI,週末の予定はなんにもないよ あのコとの約束は キャンセルだし 巷で有名な占いによれば 運気...,0


In [5]:
#. targetの確認
df["flg"].value_counts()

0    484
1     92
Name: flg, dtype: int64

In [6]:
#. artistの種類を確認
df["artist"].unique()

array(["I Don't Like Monday", 'SUPER BEAVER', 'YUKI', 'Galileo Galilei',
       '緑黄色社会', 'ELLEGADEN'], dtype=object)

In [7]:
#. artistを数値化
map_name = {"I Don't Like Monday":0, "SUPER BEAVER":1, "YUKI":2, 'Galileo Galilei':3, '緑黄色社会':4, 'ELLEGADEN':5}
map_idname = {0:"I Don't Like Monday", 1:"SUPER BEAVER'", 2:"YUKI", 3:'Galileo Galilei', 4:'緑黄色社会', 5:'ELLEGADEN'}

df["artist_true"] = df["artist"].map(map_name)
print(df["artist_true"].value_counts())
print(df["artist"].value_counts())

2    170
1    128
3     79
5     75
0     70
4     54
Name: artist_true, dtype: int64
YUKI                   170
SUPER BEAVER           128
Galileo Galilei         79
ELLEGADEN               75
I Don't Like Monday     70
緑黄色社会                   54
Name: artist, dtype: int64


In [8]:
#. test用のデータを取得
tmp = df.groupby("artist_true")
test_df = tmp.apply(lambda x: x.sample(n=10))  #. 各グループから10件取得
test_df = test_df.reset_index(level='artist_true', drop=True)  #. multiindexを解除

In [9]:
#データの確認
test_df.head(3)

,artist,song,lyrics,flg,artist_true
4,I Don't Like Monday,Up to U,珍しいね You drink a lot today 彼氏と喧嘩でもしたのかい？ 話さなくて...,0,0
40,I Don't Like Monday,ノラリ・クラリ,余計な一言のせいで また君を怒らせたんだ なんでいつもこうなっちゃうかな 自分が嫌になるね ...,0,0
44,I Don't Like Monday,Perfect Night,It's Friday Friday Friday Night 仕事切り上げ アナタの待つ場...,0,0


In [10]:
#dfからtestデータに含まれていないもののみを取得
test_df["kbn"] = 1  #.  testデータと分かるようにフラグ付け

df = pd.merge(df, test_df[["artist", "song", "kbn"]], on = ["artist", "song"], how = "left")
df_data = df[df["kbn"] != 1]
test_data = df[df["kbn"] == 1]

In [11]:
#不要な列を削除
df_data.drop("kbn", axis=1, inplace=True)
test_data.drop("kbn", axis=1, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
#データの中身を確認
df_data["lyrics"]

0      ループしてる同じ　毎日を繰り返し　まるで機械みたい　息を止め動いてる　車の中レディオ　音楽が...
1      出逢った頃とおんなじ様な　馬鹿みたく晴れた空だ　並んで歩いた緑道は　微かに夏の匂いがした　　...
2      週末の予定はなんにもないよ　あのコとの約束は　キャンセルだし　　巷で有名な占いによれば　運気...
3      You have sophisticated eyes　You make me sweete...
5      Maybe it's all right, We can dance all night　あ...
                             ...                        
571    I am lying You are lying　We keep our backs tur...
572    Wake me up before you leave　I've got an interv...
573    君の手に　上手く馴染むもの　君の目に綺麗に映るもの　それだけでいい　君の手が今も暖かく　君の...
574    She looks smiley　He looks friendly　You maybe n...
575    None of us can help dreaming　Because a dream i...
Name: lyrics, Length: 516, dtype: object

In [13]:
#文字数のカウント
df_data["len"] = df_data["lyrics"].str.len()

<ipython-input-13-081da19b45a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data["len"] = df_data["lyrics"].str.len()


In [14]:
df_data.groupby("artist_true")["len"].agg(["min","mean","max"]) 

,min,mean,max
artist_true,,,
0,389,922.416667,2175
1,183,552.338983,839
2,187,498.756250,1262
3,139,433.985507,714
4,262,518.227273,667
5,199,883.107692,1715


YUKI、ガリレオガリレイは短いと100文字代、長くても７14文字と文字数が少ない。
一方でI dont like mondayは長いのは平均が一番高く、最大値も2,000以上となっている。

英語を歌詞に使用することが多い0(i dont like monday)と5(ELLEGADEN)のスペース数が他と比べて圧倒的に多い

In [15]:
#malti_bert

In [16]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.9 MB/s eta 0:00:00


In [17]:
import torch

In [18]:
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

In [19]:
if torch.cuda.is_available():
    print("GPU")
    !nvidia-smi
else:
    print("not GPU")

GPU
Sat Jan 14 22:24:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    44W / 400W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+---

In [20]:
#トークナイザの読み込み
conf="cl-tohoku/bert-base-japanese-whole-word-masking"

!pip install fugashi
import fugashi
!pip install ipadic
import ipadic

!pip install sentencepiece
import sentencepiece

tokenizer = AutoTokenizer.from_pretrained(conf)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.9/615.9 KB 11.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 80.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=e206315417b8b240634dff7ac9a44a26a1bbc9a3d3245fc64a154c9dc3f428f5
  Stored in directory: /root/.cache/pip/wheels/45/b7/f5/a21e68db846eedcd00d69e37d60bab3f68eb20b1d99cdff652
Successfully built ipadic
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00


Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

In [21]:
#トークナイザを用いて文章を符号化
encoding = tokenizer.batch_encode_plus(
                    df_data["lyrics"].tolist(),
                    max_length = 512,
                    padding = "max_length",
                    truncation=True,
                    return_tensors="pt"
)

encoding

test_encoding = tokenizer.batch_encode_plus(
                    test_df["lyrics"].tolist(),
                    max_length = 512,
                    padding = "max_length",
                    truncation=True,
                    return_tensors="pt"
)

test_encoding

{'input_ids': tensor([[    2,  7702,  1852,  ...,     0,     0,     0],
        [    2,  1834, 28822,  ...,     0,     0,     0],
        [    2, 13668,  1507,  ...,     0,     0,     0],
        ...,
        [    2,  2203, 13954,  ...,  4067, 24347,     3],
        [    2,  9420,   480,  ...,     0,     0,     0],
        [    2,  7446,   822,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [22]:
input_ids = torch.tensor(encoding["input_ids"], dtype=torch.long)
attention_mask = torch.tensor(encoding["attention_mask"], dtype=torch.long)
test_input_ids = torch.tensor(test_encoding["input_ids"], dtype=torch.long)
test_attention_mask = torch.tensor(test_encoding["attention_mask"], dtype=torch.long)

<ipython-input-22-f133fd293fec>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(encoding["input_ids"], dtype=torch.long)
<ipython-input-22-f133fd293fec>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(encoding["attention_mask"], dtype=torch.long)
<ipython-input-22-f133fd293fec>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_input_ids = torch.tensor(test_encoding["input_ids"], dtype=torch.long)
<ipython-input-22-f133fd293fec>:4: UserWarning: To copy construct from a tensor, it

In [23]:
MAX_LEN = 512

In [24]:
train_encodings = tokenizer(df_data['lyrics'].to_list(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['lyrics'].to_list(), truncation=True, padding=True)

In [25]:
train_labels = torch.tensor(df_data["artist_true"].values)
test_labels = torch.tensor(test_df["artist_true"].values)

In [26]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#データセットクラスの作成
train_dataset = TensorDataset(input_ids, attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)

#データローダーの作成
batch_size = 32

#データローダー作成
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

In [27]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

#モデル定義
model = BertForSequenceClassification.from_pretrained(
    conf,
    num_labels = 6,
    output_attentions = False,
    output_hidden_states = False,
)

#GPUへ
model.cuda()

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
for data in train_dataloader:
  print("data[0]",data[0])
  print("data[1]",data[1])
  print("data[2]",data[2])

data[0] tensor([[    2, 10436,    15,  ...,     0,     0,     0],
        [    2,    71, 30961,  ...,     0,     0,     0],
        [    2, 10223,     5,  ...,     0,     0,     0],
        ...,
        [    2, 19241,     9,  ...,     0,     0,     0],
        [    2, 10756,   140,  ...,     0,     0,     0],
        [    2, 12250, 29182,  ..., 28666,  4752,     3]])
data[1] tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])
data[2] tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
data[3] tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
data[0] tensor([[    2, 13668,  1507,  ...,  1868, 28270,     3],
        [    2, 17012,  1507,  ..., 26873,  1462,     3],
        [    2,  1834, 28822,  ...,   

In [ ]:
#エラーが発生
#lossにstrが含まれていると記載あり、確認するとmodelの出力方法が(2.1344,loss_conda)(logit)[[0.1,0.2],[0.2,0.3]]
#のような形になっておりエラーが発生した。https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification.forward.example-4を確認
#https://teratail.com/questions/375965?rss=allに訂正コードありmodelの後ろに.lossをつけることで解決
"""
# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        #loss, logits = 
        print(model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in test_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            val_loss += loss.item()
    return val_loss
"""

In [34]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 最適化手法の設定
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
# 訓練パートの定義
max_epoch = 10
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
  model.train() # 訓練モードで実行
  train_loss = 0
  for batch in train_dataloader: # train_dataloaderはword_id, mask, labelを出力する点に注意
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      optimizer.zero_grad() #最適化された全ての勾配を０にする
      loss = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels).loss # 戻り値とここを修正
      loss.backward()  #誤差逆伝播
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) #勾配爆発の問題を軽減するために使用（勾配の最大値を決める）
      optimizer.step()
      train_loss += loss.item()

  model.eval() # 訓練モードをオフ
  val_loss = 0
  with torch.no_grad():# 勾配を計算しない
    for batch in test_dataloader:
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      with torch.no_grad():        
          loss = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels).loss # 戻り値とここを修正
          val_loss += loss.item()
  ave_train_loss = train_loss / len(train_dataloader)
  ave_val_loss = val_loss / len(test_dataloader)
  print("epoch: {}, train_loss: {}, val_loss: {}".format(epoch, ave_train_loss, ave_val_loss))


epoch: 0, train_loss: 1.6900082861675936, val_loss: 1.6256357431411743
epoch: 1, train_loss: 1.4067065645666683, val_loss: 1.4324563145637512
epoch: 2, train_loss: 1.2026958605822395, val_loss: 1.4011138677597046
epoch: 3, train_loss: 1.071692003923304, val_loss: 1.2166653275489807
epoch: 4, train_loss: 1.0152204545105206, val_loss: 1.2597314417362213
epoch: 5, train_loss: 0.8422667550690034, val_loss: 1.2230782508850098
epoch: 6, train_loss: 0.8278782700791079, val_loss: 1.4877294301986694
epoch: 7, train_loss: 0.7035239623749957, val_loss: 1.0472650527954102
epoch: 8, train_loss: 0.6788694781853872, val_loss: 1.185939610004425
epoch: 9, train_loss: 0.5124410000794074, val_loss: 1.1082004308700562


In [35]:
model.eval()# 訓練モードをオフ
for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

In [36]:
## 予測結果の確認
print(f'出力:{preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[-0.3689, -1.4857,  0.9313,  1.4392, -0.4201, -0.5509],
        [ 0.0964, -0.3080, -1.5573,  2.4974,  0.6108, -1.0064],
        [-0.0858, -1.7209,  0.2342,  1.6981, -0.3660, -1.1093],
        [-0.0621, -0.8135, -0.9473,  1.7681, -0.1605, -0.3952],
        [-0.1672, -1.7691,  2.0648,  0.8734, -0.5709, -1.2465],
        [-0.4127,  0.2437, -1.1870,  1.2373,  0.0737, -0.6636],
        [-0.1335, -1.7579,  0.9722,  1.9102, -0.2541, -1.0127],
        [-0.2580, -0.8880, -0.7828,  2.1514,  0.2282, -1.1922],
        [-1.0667,  2.5868, -2.0736,  0.3600,  0.5345, -1.0844],
        [-0.2858, -0.9988,  0.7288,  1.3541, -0.0644, -1.8197],
        [-0.5031,  1.1978, -1.6297,  0.9685,  0.1374, -0.5643],
        [-0.2227,  0.4750, -2.5246,  1.6550,  0.5405, -0.7173],
        [-1.0535,  3.7812, -1.8701, -1.2303, -0.0183, -0.6171],
        [-0.1872,  1.1848, -1.1751,  0.2164, -0.0384, -0.0723],
        [-0.1139, -1.1040, -0.7088,  2.1013, -0.0475, -1.2

In [37]:
# 比較しやすい様にpd.dataframeへ整形
import pandas as pd
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1',"logit_2","logit_3","logit_4","logit_5"])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

,logit_0,logit_1,logit_2,logit_3,logit_4,logit_5,pred_label,true_label
0,-0.368910,-1.485688,0.931261,1.439242,-0.420126,-0.550916,3,3
1,0.096366,-0.308044,-1.557329,2.497411,0.610827,-1.006361,3,3
2,-0.085829,-1.720940,0.234175,1.698121,-0.365973,-1.109340,3,3
3,-0.062074,-0.813453,-0.947264,1.768128,-0.160469,-0.395184,3,3
4,-0.167206,-1.769142,2.064774,0.873396,-0.570931,-1.246452,2,3


In [38]:
accuracy_df

,logit_0,logit_1,logit_2,logit_3,logit_4,logit_5,pred_label,true_label
0,-0.368910,-1.485688,0.931261,1.439242,-0.420126,-0.550916,3,3
1,0.096366,-0.308044,-1.557329,2.497411,0.610827,-1.006361,3,3
2,-0.085829,-1.720940,0.234175,1.698121,-0.365973,-1.109340,3,3
3,-0.062074,-0.813453,-0.947264,1.768128,-0.160469,-0.395184,3,3
4,-0.167206,-1.769142,2.064774,0.873396,-0.570931,-1.246452,2,3
5,-0.412687,0.243663,-1.187045,1.237338,0.073712,-0.663644,3,3
6,-0.133486,-1.757914,0.972221,1.910229,-0.254120,-1.012743,3,3
7,-0.257993,-0.887987,-0.782841,2.151414,0.228161,-1.192164,3,3
8,-1.066705,2.586761,-2.073586,0.360000,0.534474,-1.084377,1,4
9,-0.285841,-0.998789,0.728835,1.354091,-0.064359,-1.819664,3,4


In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(pred_df, label_df)

0.6071428571428571